<a href="https://colab.research.google.com/github/Mpr9640/FairnessIndicator/blob/main/FairIndicator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#installing the libraries needed in this model
#! is for to run the commands in notebook
!pip install --quiet --upgrade\
     tensorflow-model-analysis\
     tensorflow-model-remediation \
     fairness-indicators

In [ ]:
# import the libraries
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_model_analysis as tfma
import tensorflow_model_remediation as tfmr
from tensorflow_model_remediation import min_diff
import google.protobuf  # used to take some data from unserialized dataset
from google.protobuf import text_format # for serializing the data( means sequencing the data)

In [ ]:
acs_df=pd.read_csv(filepath_or_buffer='https://download.mlcc.google.com/mledu-datasets/acsincome_raw_2018.csv')
acs_df.sample(5)

,AGEP,COW,SCHL,MAR,OCCP,POBP,RELP,WKHP,SEX,RAC1P,ST,PINCP
1285044,21.0,1.0,21.0,5.0,4030.0,42.0,0.0,30.0,2.0,1.0,42.0,10000.0
991998,40.0,1.0,20.0,1.0,7210.0,36.0,1.0,45.0,1.0,1.0,36.0,57000.0
334967,38.0,1.0,22.0,1.0,1108.0,210.0,0.0,40.0,1.0,6.0,12.0,63000.0
1257732,54.0,1.0,16.0,5.0,8255.0,42.0,13.0,40.0,1.0,1.0,42.0,43000.0
350620,41.0,1.0,24.0,1.0,3090.0,12.0,1.0,60.0,2.0,1.0,12.0,476000.0


In [ ]:
acs_df['PINCP'].describe()


,PINCP
count,1.664500e+06
mean,5.666386e+04
std,7.306745e+04
min,1.040000e+02
25%,2.000000e+04
50%,3.900000e+04
75%,6.800000e+04
max,1.423000e+06


In [ ]:
#converting the label into binary representation
label_key='PINCP'
label_threshold = 50000.00
acs_df[label_key]=acs_df[label_key].apply(
    lambda income: 1 if income > label_threshold else 0)
acs_df.sample(20)

,AGEP,COW,SCHL,MAR,OCCP,POBP,RELP,WKHP,SEX,RAC1P,ST,PINCP
241499,60.0,2.0,22.0,1.0,630.0,303.0,1.0,40.0,1.0,1.0,6.0,1
905695,30.0,6.0,18.0,1.0,6130.0,33.0,16.0,60.0,1.0,1.0,33.0,0
1187244,20.0,1.0,18.0,5.0,7315.0,39.0,0.0,40.0,1.0,1.0,39.0,0
128556,51.0,1.0,11.0,1.0,6260.0,303.0,0.0,50.0,1.0,8.0,6.0,0
61402,57.0,1.0,19.0,1.0,5000.0,15.0,0.0,40.0,2.0,1.0,5.0,1
1312977,58.0,1.0,16.0,1.0,5240.0,45.0,0.0,30.0,2.0,1.0,45.0,0
440754,66.0,1.0,21.0,1.0,800.0,36.0,0.0,40.0,1.0,1.0,13.0,0
1430092,54.0,7.0,23.0,1.0,3090.0,48.0,0.0,48.0,1.0,1.0,48.0,1
888358,61.0,1.0,18.0,5.0,7340.0,46.0,0.0,40.0,1.0,1.0,31.0,1
334851,56.0,1.0,16.0,1.0,5240.0,12.0,0.0,40.0,2.0,1.0,12.0,0


In [ ]:
inputs = {}
feautures=acs_df.copy()
feautures.pop(label_key)
# instantiate a keras node for each column in the dataset.
for name, column in feautures.items():
  if name !='label_key':
    inputs[name]=tf.keras.Input(shape=(1,),name=name,dtype=tf.float64)
#stack the inputs as a dictionary and preprocess them
def stack_dic(inputs, fun=tf.stack): #fun is a flexible function to use different stacking function.
  values=[]
  for key in sorted(inputs.keys()): # sorted is used to maintain correct order in dictonary
    values.append(tf.cast(inputs[key],tf.float64))
  return fun(values,axis=-1)  #-1 defines the last axis
x=stack_dic(inputs,fun=tf.concat)

normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(stack_dic(dict(feautures))) # adapt is used to find mean and variance

x=normalizer(x)
x=tf.keras.layers.Dense(64,activation='relu')(x)
x=tf.keras.layers.Dense(32,activation='relu')(x)
x=tf.keras.layers.Dense(1,activation='sigmoid')(x)
base_model=tf.keras.Model(inputs=inputs,outputs=x)


In [ ]:
#because of binary classification,cross-entropy loss is workable.
#defining metrics to observe model performance while training
metrics=[
    tf.keras.metrics.BinaryAccuracy(name='accuracy'),
    tf.keras.metrics.AUC(name='auc'),
]
base_model.compile(optimizer='adam',loss='binary_crossentropy',metrics=metrics)

In [ ]:
# to convert the pandas dataframe into tensor.datset whihc is required to work with tensor dataset.
def dataframe_to_dataset(dataframe):
  dataframe=dataframe.copy()
  labels=dataframe.pop(label_key)
  dataset=tf.data.Dataset.from_tensor_slices((dict(dataframe),labels))
  dataset=dataset.shuffle(buffer_size=len(dataframe))
  return dataset

In [ ]:
#training the model and using sample to split the dataset.
RandomState=np.random.RandomState(42)
batch_size=50
epochs=3
acs_train_df=acs_df.sample(frac=0.8,random_state=RandomState)
acs_train_ds=dataframe_to_dataset(acs_train_df)
acs_train_batches=acs_train_ds.batch(batch_size)
base_model.fit(acs_train_batches,epochs=epochs)


NameError: name 'np' is not defined

In [ ]:
# Evaluating the model with remaining dataset.
acs_test_df=acs_df.drop(acs_train_df.index)
acs_test_ds=dataframe_to_dataset(acs_test_df)
acs_test_batches=acs_test_ds.batch(BatchSize)
base_model.evaluate(acs_test_batches, batch_size=BatchSize)


NameError: name 'acs_df' is not defined

In [ ]:
#predicting the output values
base_model_prediction=base_model.predict(acs_test_batches,batch_size=BatchSize)


In [ ]:
sensitive_attribute_values={1.0: "Male", 2.0: "Female"}
sensitive_attribute_key="SEX"
prediction_key="Pred"
base_model_analysis=acs_test_df.copy()
base_model_analysis[sensitive_attribute_key].replace(sensitive_attribute_values,inplace=True)
#inplace=True, returns the dataframe without creating a new dataset.
base_model_analysis[prediction_key]=base_model_prediction
base_model_analysis.sample(10)

In [ ]:
#Configure Fairness Indicators
eval_config_text= """
    model_specs{
      prediction_key: "%s"
      label_key: "%s"
    }
    metrics_specs{
      metrics{class_name: "ExampleCount"}
      meetrics{class_name: " BinaryAccuracy"}
      metrics{class_name: "AUC"}
      metrics{
        class_name: "FairnessIndicators"
        config: {
          thresholds: [0.5]

      }

    }
    slicing_specs{
      feature_keys: "%s"
    }
    """ % (prediction_key,label_key,sensitive_attribute_key)
eval_config=text_format.Parse(eval_config_text,tfma.EvalConfig())
#run tensorflow model analysis
base_model_eval_result=tfma.analyze_raw_data(
    base_model_analysis,
    eval_config=eval_config,
)

In [ ]:
#renseing fairness indicators
tfma.addons.fairness.view.widget_view.render_fairness_indicatir(base_model_eval_result)

In [ ]:
sensitive_group_pos = acs_train_df[(acs_train_df[sensitive_attribute_key]==2.0) & (acs_train_df[label_key]==1) ]
non_sensitive_group_pos=acs_train_df[(acs_train_df[sensitive_attribute_key]==1.0) &
                                     (acs_train_df[label_key]==1)]
print(len(sensitive_group_pos), 'positively labeled sensitive examples')
print(len(non_sensitive_group_pos), 'positively labeled on non-sensitive examples')


In [ ]:
#converting sensitive and non-sensitive subsets into tf.data.dataset
min_diff_batch_size=50
sensitive_group_ds=dataframe_to_dataset(sensitive_group_pos)
non_sensitive_group_ds=dataframe_to_dataset(non_sensitive_group_pos)
#batch the subsets
sensitivse_group_batches=sensitive_group_ds.batch(min_diff_batch_size, drop_remainder=True)
non_sensitive_group_batches=non_sensitive_group_ds.batch(min_diff_batch_size, drop_remainder=True)


In [ ]:
acs_train_min_diff_ds=min_diff.keras.utils.pack_min_diff_data(
    original_dataset=acs_train_batches,
    sensitive_group_dataset=sensitive_group_batches,
    non_sensitive_group_dataset=non_senstive_group_batches
)

In [ ]:
#wrap the original model into min_diff Model
min_diff_model= min_diff.keras.min_diff_model(original_model=base_model,
                                              loss=min_diff.losses.MMDLOSS(),
                                              loss_weight=1)
min_diff_model.complie(optimizer='adam',
                       loss=tf.keras.losses.binary_crossentropy(from_logits=False),
                       metrics=metrics)

In [ ]:
#training the model with packed data instead of original train dataset.
min_diff_model.fit(acs_train_min_diff_ds,epochs=epochs)

In [ ]:
#evaluating min_diff model
min_diff_model.evaluate(acs_test_batches,batch_size=batch_size)


In [ ]:
min_diff_model_prediction=min_diff_model.predict(acs_test_batches,batch_size=batch_size)

In [ ]:
#Appending the prediction column into the dataframe and than passing into the fairness indicaators for evaluation.
min_diff_model_analysis=acs_test_df.copy()
min_diff_model_analysis[sensitive_attribute_key].replace(senstive_attribute_values,inplace=True)
min_diff_model_analysis[prediction_key]=min_diff_model_predictions
min_diff_model_eval_result=tfma.analyze_raw_data(min_diff_model_analysis, eval_config)

In [ ]:
#Reviewing MinDiff results
tfma.addon.fairness.view.widget_view.render_fairness_indicator(min_diff_model_eval_result)